This code aims to authenticate GEE, read in the desired asset, and open as an image. 
For this example we will be using MapBiomas Collection 9 [Data](https://brasil.mapbiomas.org/en/colecoes-mapbiomas/) to isolate the Atlantic Forest of Brazil.

In order to run this, you will need a Google Earth Engine (GEE) account in order to authenticate and call an asset. Here are some resourfes for getting started and troubleshooting: 

Basic account set up guide: [Link](https://courses.spatialthoughts.com/gee-sign-up.html)
Transition to cloud projects: [Link](https://developers.google.com/earth-engine/guides/transition_to_cloud_projects)
Data Usage Guide: [Link](https://developers.google.com/earth-engine/guides/usage)
Create Access Credentials: [Link](https://developers.google.com/workspace/guides/create-credentials#api-key)
Authentication and Initialization: [Link](https://developers.google.com/earth-engine/guides/auth)
Authentication in VSCode: [Link](https://stackoverflow.com/questions/78374548/no-authentication-box-appears-when-authenticating-google-earth-engine-gee-pyth)

# Importing GEE Asset

1. Authenticate GEE
2. Read the asset
3. Open as an Image
Additional steps:
* This example selects specific years, filter for forest land cover, and mask to the boundaries of the Atlantic Forest.
4. Select the band (using year)
5. Filter the class value (forest formation)
6. Mask to biome (Atlantic Forest)
7. Reporject to target CRS

In [ ]:
# Import Packages
import os
#import numpy as np
#import pylandstats as pls
#from scipy import stats
#import dask.array as da
#import glob
import xarray as xr
#import rioxarray as rxr
#import pandas as pd
#import re
#from pathlib import Path
#import matplotlib.pyplot as plt

# GEE
import ee
import geemap

In [ ]:
# GEE Functions
def authenticate_gee():
    """
    AuthenticThis function attempts run the authentification process for GEE.
    User must sign in with their credentials and make sure it is properly set up. (See readme for helpful links)

    Returns:
    gee.Map or None: A GEE Map object if authentication is successful,
                     None otherwise.
    """
    import ee
    import geemap
    # Initialize GEE
    ee.Authenticate()
    ee.Authenticate(auth_mode='notebook') # Needed for Jupyter notebook
    ee.Initialize()

In [ ]:
authenticate_gee()

In [ ]:
# Asset list
assetInput = "projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1"
assetBiomes = "projects/mapbiomas-workspace/AUXILIAR/ESTATISTICAS/biomas-5"

In [ ]:
# Read Image
input = ee.Image(assetInput)
print(input)

# Read Biome
biomes = ee.Image(assetBiomes)
print(biomes)

In [ ]:
# Read in all data to 1 image, clip, and convert to xarray

years = [2019, 2020, 2021, 2022, 2023]
# select only specific years and forest formation level
mb_images = [input.select(f'classification_{year}').eq(3) for year in years]
# convert multiple images to a collection
mb_collection = ee.ImageCollection.fromImages(mb_images)

# Mask data extent to Atlantic Forest biome asset 
biome_mask = biomes.eq(4)
# Apply mask to each iamge in the collection 
mb_clipped = mb_collection.map(lambda image: image.updateMask(biome_mask))

# Convert ee.ImageCollection to xarray
mb_xarray = geemap.ee_to_xarray(
    mb_clipped,
    crs='EPSG:6618',
    scale=30,
    geometry=mb_clipped.first().geometry()
)

# Rename data variable names
mb_xarray = mb_xarray.rename_vars({'classification_2019': 'data_vars'})

# Chnage time dimension to associated year
mb_xarray = mb_xarray.assign_coords(
    time = ('time', years)
    )

# look at data structure
mb_xarray